In [2]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
    "bolt://localhost:7687",
    auth=basic_auth(user="neo4j", password="neo4j1234")
)

In [5]:
# 사용자가 좋아하는 영화($favorite)을 찾고 그 영화에 출연한 배우(actor)가 출연한 다른 영화(rec)을 중복없이 10개 나열
cypher_query = '''
MATCH (movie:Movie {title:$favorite})<-[:ACTED_IN]-(actor)-[:ACTED_IN]->(rec:Movie)
RETURN distinct rec.title as title LIMIT 10
'''

with driver.session(database="neo4j") as session:
    results = session.execute_read( # 읽기 전용의 트랜잭선 실행
        lambda tx: tx.run(cypher_query, favorite="The Matrix").data())
    for record in results:
        print(record['title'])

C:\Users\kidong\AppData\Local\Temp\ipykernel_12460\2225431544.py:6: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:


Cloud Atlas
V for Vendetta
The Matrix Revolutions
The Matrix Reloaded
The Replacements
Johnny Mnemonic
Something's Gotta Give
The Devil's Advocate


In [10]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver(
    "bolt://localhost:7687",
    auth=basic_auth(user="neo4j", password="neo4j1234")
)

In [14]:
# Movie의 타이틀이 'Top Gun'인 Movie(m)에 출연한 사람(p)의 이름(p.name)을 출력
result = driver.execute_query('''
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title = 'Top Gun'
RETURN p.name
''')
print(result) # records 리스트로 반환된 구조

EagerResult(records=[<Record p.name='Val Kilmer'>, <Record p.name='Meg Ryan'>, <Record p.name='Tom Skerritt'>, <Record p.name='Kelly McGillis'>, <Record p.name='Tom Cruise'>, <Record p.name='Anthony Edwards'>], summary=<neo4j._work.summary.ResultSummary object at 0x0000011D8AC66C20>, keys=['p.name'])


In [13]:
for record in result.records:
    print(record["p.name"])

Val Kilmer
Meg Ryan
Tom Skerritt
Kelly McGillis
Tom Cruise
Anthony Edwards


In [23]:
from langchain_community.graphs.neo4j_graph import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question}"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [24]:
CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
Here is an example:

Question: Which managers own Neo4j stocks?
Context:[manager:CTL LLC, manager:JANE STREET GROUP LLC]
Helpful Answer: CTL LLC, JANE STREET GROUP LLC owns Neo4j stocks.

Follow this example when generating answers.
If the provided information is empty, say that you don't know the answer.
Information:
{context}

Question: {question}
Helpful Answer:"""
CYPHER_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

In [29]:
graph = Neo4jGraph(url="bolt://localhost:7687", username='neo4j', password='neo4j1234')

In [30]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY),
    graph=graph,
    verbose=True
)

In [31]:
chain.run("Who played in Top Gun?")

C:\Users\kidong\AppData\Local\Temp\ipykernel_12460\1697278274.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  chain.run("Who played in Top Gun?")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[{'p.name': 'Val Kilmer'}, {'p.name': 'Meg Ryan'}, {'p.name': 'Tom Skerritt'}, {'p.name': 'Kelly McGillis'}, {'p.name': 'Tom Cruise'}, {'p.name': 'Anthony Edwards'}]

> Finished chain.


'Val Kilmer, Meg Ryan, Tom Skerritt, Kelly McGillis, Tom Cruise, Anthony Edwards played in Top Gun.'

In [32]:
chain.run("How many people played in Top Gun?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person)-[r:ACTED_IN]->(:Movie {title: 'Top Gun'})
RETURN COUNT(r)
Full Context:
[{'COUNT(r)': 6}]

> Finished chain.


'6 people played in Top Gun.'